## Import packages

In [6]:
from pathlib import Path
import os
from utils import check_integrity

## Data Integrity Check

Note: The number of all supported coins: 169

In [9]:
data_dir = Path(os.getcwd(), '..', 'data').resolve()
intervals = ['1D', '1W', '3h', '30m']
coins_pass_check_1_3 = {}   # "2021-01-13" - "2021-03-14"
coins_pass_check_8_10 = {}  # "2021-08-15" - "2021-10-16"

for interval in intervals:
    coins_pass_check_1_3[interval] = []
    coins_pass_check_8_10[interval] = []
    data_interval_dir = Path(data_dir, interval)
    files_path_list = [p for p in data_interval_dir.iterdir()]
    for file_path in files_path_list:
        coin_code = file_path.name.split("_")[0]
        if check_integrity("2021-01-12", "2021-03-15", file_path) is not None:
            coins_pass_check_1_3[interval].append(coin_code)
        if check_integrity("2021-08-15", "2021-10-16", file_path) is not None:
            coins_pass_check_8_10[interval].append(coin_code)


In [10]:
coins_pass_check_1_3

{'1D': ['BTC',
  'ETH',
  'AVAX',
  'LTC',
  'ZEC',
  'TRX',
  'DOT',
  'XRP',
  'YFI',
  'MKR',
  'XAUT',
  'BCHN',
  'EGLD',
  'KSM',
  'COMP',
  'XMR',
  'DAI',
  'BSV',
  'MLN',
  'FIL',
  'BTG',
  'FTT',
  'ETC',
  'NEO',
  'LINK',
  'REP',
  'BAL',
  'UNI',
  'QTM',
  'BAND',
  'OMG',
  'SNX',
  'ANT',
  'GTX',
  'HEZ',
  'XTZ',
  'EOS',
  'ENJ',
  'LEO',
  'LRC',
  'UOS',
  'KNC',
  'BAT',
  'ADA',
  'ZRX',
  'PAX'],
 '1W': [],
 '3h': ['BTC',
  'ETH',
  'AVAX',
  'LTC',
  'ZEC',
  'TRX',
  'DOT',
  'XRP',
  'YFI',
  'XAUT',
  'BCHN',
  'EGLD',
  'KSM',
  'COMP',
  'XMR',
  'DAI',
  'BSV',
  'FIL',
  'BTG',
  'ETC',
  'NEO',
  'LINK',
  'BAL',
  'UNI',
  'BAND',
  'OMG',
  'SNX',
  'HEZ',
  'XTZ',
  'EOS',
  'LEO',
  'UOS',
  'BAT',
  'ADA',
  'ZRX',
  'PAX'],
 '30m': ['BTC',
  'ETH',
  'AVAX',
  'LTC',
  'ZEC',
  'TRX',
  'DOT',
  'XRP',
  'YFI',
  'XAUT',
  'BCHN',
  'EGLD',
  'COMP',
  'XMR',
  'BSV',
  'FIL',
  'ETC',
  'NEO',
  'LINK',
  'BAL',
  'UNI',
  'BAND',
  'OMG',
  

## Time Series Visualization

In [13]:
data_interval_dir = Path(data_dir, '1D')

In [19]:
[i for i in data_interval_dir.iterdir()][0].name.split("_")[0]

'AVAX'